In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import wget

In [2]:
def load_data(filePath):
    df = pd.read_csv(filePath)
    return df

## Load CSV-data into DataFrames

In [3]:
# Prepped Time Series Data with datapoints for each date, for each country
PREPPED_DATA = "output/COVID9-cleaned_03-05-2020_21:13.csv"
complete_df = load_data(PREPPED_DATA)

# Prepped data per country
COUNTRY_DATA = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv"
country_df = pd.read_csv(COUNTRY_DATA)
# Rename columns
country_df.rename(columns={'Country_Region':'Country', 'Last_Update':'Date', 'Long_':'Long'}, inplace=True)
# Rename United Kingdom to UK
country_df['Country'] = country_df['Country'].replace('United Kingdom', 'UK')
# Fetch the desired columns
country_df = country_df[['Country', 'Date', 'Lat', 'Long', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'UID', 'ISO3']]

## Prepare DataFrames for plotting with Plotly

In [77]:
top_countries = country_df.sort_values('Confirmed', ascending=False)['Country'].to_list()


# Create a new DataFrame for the chosen countries
def top_countries_df(n, init_on=100):
    df = complete_df[pd.DataFrame(complete_df['Country'].tolist()).isin(top_countries[:n]).any(1)]
    return df[df['Confirmed'] >= init_on]
# Sort out data from before 100 Confirmed cases #top_countries_df[top_countries_df['Confirmed'] >= 100] 
top_countries_100_df = top_countries_df(5)

# norge = complete_df[complete_df['Country'] == 'Norway']
# norge = norge[norge['Confirmed'] >= 100]

In [5]:
n=10
sorted_country_df = country_df.sort_values('Confirmed', ascending=False).head(n)

fig = px.scatter(sorted_country_df, 
                x="Country", y="Deaths",
                size="Confirmed", color="Country",
                hover_name="Country", size_max=80,
                color_discrete_sequence=px.colors.qualitative.G10)

fig.update_layout(
title=f"Top {n} worst hit countries",
xaxis_title="Countries",
yaxis_title="Deaths",
)

fig.update_layout(
    autosize=False,
    width=800,
    height=450,
    plot_bgcolor='white')


fig.show()

In [6]:
px.set_mapbox_access_token(open(".mapbox_token").read())

df = country_df
fig = px.scatter_geo(
    df, 
    #lat="Lat", lon="Long",
    locations="ISO3",
    color="Deaths", size="Confirmed",
    size_max=40,
    #color_continuous_scale="blues",
    #reversescale=True,
    projection="natural earth",
    template='ggplot2'
    )
# "open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" or "stamen-watercolor" yeild maps composed of raster tiles from various public tile servers which do not  require signups or access tokens

# "basic", "streets", "outdoors", "light", "dark", "satellite", or "satellite-streets" yeild maps composed of vector tiles from the Mapbox service, and do require a Mapbox Access Token or an on-premise Mapbox installation.

# https://plotly.com/python/builtin-colorscales/
# https://plotly.com/python/discrete-color/

# https://plotly.com/python/mapbox-layers/


fig.update_layout(
    title="Global deathcount",
    title_x=0.0,
    autosize=False,
    width=800,
    height=450,
)


fig.write_image("images/earth_deaths.pdf")

fig.show()

In [7]:
px.set_mapbox_access_token(open(".mapbox_token").read())

df = country_df
fig = px.scatter_mapbox(
    df, lat="Lat", lon="Long",
    color="Deaths", size="Confirmed",
    size_max=40,
    zoom=0,
    opacity=0.9,
    template='ggplot2'
    #color_continuous_scale="blues"
    )
# "open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" or "stamen-watercolor" yeild maps composed of raster tiles from various public tile servers which do not  require signups or access tokens

# "basic", "streets", "outdoors", "light", "dark", "satellite", or "satellite-streets" yeild maps composed of vector tiles from the Mapbox service, and do require a Mapbox Access Token or an on-premise Mapbox installation.

# https://plotly.com/python/builtin-colorscales/
# https://plotly.com/python/discrete-color/

# https://plotly.com/python/mapbox-layers/


fig.update_layout(
    autosize=False,
    width=800,
    height=450,
    mapbox_style='basic',
)


#fig.write_image("images/.pdf")

fig.show()

In [183]:
df = country_df
'''
fig = px.choropleth(df, locations="ISO3",
                    color="Deaths", 
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.Plasma

)
fig.show() '''

fig = go.Figure(
    data=go.Choropleth(
        locations = df['ISO3'],
        z = df['Deaths'],
        text = df['Country'],
        colorscale = 'Blues',
        autocolorscale=False,
        reversescale=True,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_title = 'Deaths',
        )
)

fig.update_layout(
    title='Global deathcount',
    width=800,
    height=450,
    geo = dict(
        scope='world',
        projection=go.layout.geo.Projection(type='natural earth'),
        #showocean=True,
        #oceancolor='rgb(192,192,192)'
    )
)

fig.write_image("images/choropleth_map.pdf")

#['equirectangular', 'mercator', 'orthographic', 'natural
#           earth', 'kavrayskiy7', 'miller', 'robinson', 'eckert4',
#           'azimuthal equal area', 'azimuthal equidistant', 'conic
#           equal area', 'conic conformal', 'conic equidistant',
#           'gnomonic', 'stereographic', 'mollweide', 'hammer',
#           'transverse mercator', 'albers usa', 'winkel tripel',
#           'aitoff', 'sinusoidal'#
fig.show()


In [106]:
df = country_df.sort_values('Deaths', ascending=False).head(10)

fig = px.scatter_3d(
    df, x="Country", y="Confirmed", z="Deaths",
    color="Deaths", size="Confirmed", size_max=125, hover_name="Country",
    color_continuous_scale="Thermal", opacity=0.90,
)

fig.update_layout(
    title="Worst hit countries w.r.t. deaths",
    autosize=False,
    width=800,
    height=800,
    template='plotly_white',
)

fig.update_layout(
    scene_aspectmode='manual',
    scene_aspectratio=dict(x=4, y=2, z=2),
    scene = dict(
        xaxis=dict(
            tickmode='linear'
        )
    ),
    scene_camera = dict(
        eye=dict(x=4, y=6.1, z=2.5)
    )
)


#fig.write_image("images/worst_hit_deaths_3d.pdf")

fig.show()

In [10]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

import plotly.graph_objects as go

def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure()
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 30:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 30:].columns))
            y_data = np.sum(np.asarray(df[df['Country/Region'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ))
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='Number of Confirmed Cases',
        plot_bgcolor='white'
    )
    
    fig.update_yaxes(type="log")
    fig.show()

In [11]:
#plot_cases_of_a_country("Norway")

In [134]:
df = country_df.sort_values('Confirmed', ascending= False).head(10) 

fig = go.Figure(
    data = [
        go.Bar(
            x=df['Country'],
            y=df['Confirmed'],
            name='Confirmed',
            showlegend=False,
            marker = {
                'color': px.colors.qualitative.G10
            }
        ),
        go.Scatter(
            x=df['Country'],
            y=df['Deaths'],
            name='Deaths'
        )
    ],
    layout = go.Layout(
        barmode='stack',
        title='Confirmed Cases'
    ),
)

fig.update_xaxes(title='Country')
fig.update_yaxes(title='Cases',type='log')


fig.show()

In [39]:
df = top_countries_df(10)

fig = px.line(df, x='Date', y='Confirmed', color='Country', line_group='Country', 
title='Confirmed cases comparison')

fig.update_layout(
        xaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=14)
        ))

#fig.update_layout(xaxis_rangeslider_visible=True)

fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    plot_bgcolor='white')

fig.update_yaxes(type="linear")

fig.write_image("images/Confirmed_comparison_linear10.pdf")
fig.show()


In [164]:
df = country_df.sort_values('Confirmed', ascending= False).head(10)

#df = top_countries_df(5)

fig = px.bar(df, x='Country', y='Confirmed', color='Country', color_discrete_sequence=px.colors.qualitative.G10)

fig.update_layout(
    title="Daily New Cases",
    autosize=False,
    width=800,
    height=450,
    plot_bgcolor='white')

fig.update_yaxes(type="linear")
fig.show()

In [168]:
import plotly.express as px

df = top_countries_df(10)

fig = px.scatter(df, x="Date", y="Country", size="New Cases", color="Country", size_max=40)

fig.update_layout(
title="Daily New Cases",
xaxis_title="Date",
yaxis_title="Conutry",
)

fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    plot_bgcolor='white'
)

#fig.write_image("images/daily_new_cases10.pdf")


fig.show()

In [182]:
df = top_countries_df(n=10)

fig = go.Figure(data=go.Scatter3d(
    x=df['Date'],
    y=df['Country'],
    z=df['Active'],
    text=df['Country'],
    mode='markers',
    marker=dict(
        size=df['New Cases'],
        sizeref=500,
        sizemode='diameter',
        colorscale = 'Thermal',
        color = df['New Cases'],
        colorbar_title = 'Daily New Cases',
        line_color='rgb(128, 128, 128)'
        )
    )
)


fig.update_layout(
    title = 'Active Cases Over Time',
    width=800, height=800, 
    scene_aspectmode='manual',
    scene_aspectratio=dict(x=4, y=3, z=2),
    scene = dict(
        xaxis=dict(title='Date'),
        yaxis=dict(title='Country'),
        zaxis=dict(title='Active Cases')
        ),
    scene_camera = dict(
        eye=dict(x=5, y=5.5, z=5)
    ),
    scene_zaxis_type="log"
)

#fig.write_image("images/Confirmed_comparison_linear10.pdf")

fig.show()

In [102]:
df = top_countries_df(50)

fig = px.treemap(df, path=['Country'], values='Confirmed', color_discrete_sequence=px.colors.qualitative.G10
)

fig.update_layout(
    title='Treemap for Top 10 Countries: Confirmed',
    width=800,
    height=450,
    plot_bgcolor='white',
)

#fig.savefig('images/treechart_confirmed10.pdf')

fig.show()